# 🧹 Correções – Pipeline de KPIs (Parte 1–3)

Este notebook foi gerado como **versão corrigida** para:
1) Padronizar a coluna de data (`day` → `date`).
2) Agregar os KPIs **por dia e `entity` (PF/PJ)**.
3) Calcular **TPV, Transactions, Average_Ticket** e **média móvel de 7 dias (por entidade)**.
4) Salvar o arquivo **`data/kpis_gold.csv`** pronto para o app Streamlit.

Caso você deseje incluir mais dimensões (ex.: `product`, `payment_method`), basta adicioná-las no `groupby`.


In [1]:

# =========================
# 0) Imports e Config
# =========================
from pathlib import Path
import pandas as pd

# Caminho do CSV original (ajuste se necessário)
RAW_PATH = Path("Operations_analyst_data.csv")

# Caminho de saída
OUT_DIR = Path("streamlit/data")
OUT_FILE = OUT_DIR / "kpis_gold.csv"

print("RAW_PATH:", RAW_PATH.resolve())
print("OUT_FILE:", OUT_FILE.resolve())


RAW_PATH: C:\Users\lucas\OneDrive\Área de Trabalho\Projetos\CloudWalk-Case\Operations_analyst_data.csv
OUT_FILE: C:\Users\lucas\OneDrive\Área de Trabalho\Projetos\CloudWalk-Case\streamlit\data\kpis_gold.csv


In [2]:

# =========================
# 1) Carregar dados brutos
# =========================
# Observação: Este notebook supõe que o CSV original esteja disponível em data/transactions.csv
# Se o arquivo tiver outro nome/rota, ajuste RAW_PATH acima.

df = pd.read_csv(RAW_PATH)

# Padronizar nome e tipo da coluna de data
date_col_candidates = [c for c in df.columns if c.lower() in ("day", "date", "transaction_date")]
if not date_col_candidates:
    raise ValueError("Não encontrei coluna de data (day/date/transaction_date) no CSV. Verifique o schema.")
date_col = date_col_candidates[0]

df = df.rename(columns={date_col: "date"})
df["date"] = pd.to_datetime(df["date"])

# Checagens básicas
expected_cols = ["entity","amount_transacted","quantity_transactions"]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colunas obrigatórias ausentes no CSV: {missing}. "
                     "Certifique-se de que o dataset do case foi carregado corretamente.")
print("Schema após padronização:", df.columns.tolist())


Schema após padronização: ['date', 'entity', 'product', 'price_tier', 'anticipation_method', 'payment_method', 'installments', 'amount_transacted', 'quantity_transactions', 'quantity_of_merchants']


In [3]:

# =========================
# 2) Agregar KPIs por DIA + ENTITY
# =========================
# Se quiser incluir outras dimensões (product, payment_method etc.), adicione-as no groupby.

group_dims = ["date", "entity"]

daily_kpis = (
    df.groupby(group_dims, as_index=False)
      .agg(
          TPV=("amount_transacted", "sum"),
          Transactions=("quantity_transactions", "sum")
      )
)

# KPIs derivados
daily_kpis["Average_Ticket"] = daily_kpis.apply(
    lambda r: (r["TPV"] / r["Transactions"]) if r["Transactions"] else 0.0, axis=1
)

# Atributos temporais
daily_kpis["weekday"] = daily_kpis["date"].dt.day_name()

# Ordenação e média móvel por entidade
daily_kpis = daily_kpis.sort_values(group_dims)
daily_kpis["TPV_MA7"] = daily_kpis.groupby("entity")["TPV"].transform(lambda s: s.rolling(7, min_periods=1).mean())

daily_kpis.head()


,date,entity,TPV,Transactions,Average_Ticket,weekday,TPV_MA7
0,2025-01-01,PF,1.954255e+07,128784,151.746688,Wednesday,1.954255e+07
1,2025-01-01,PJ,3.244821e+07,633255,51.240353,Wednesday,3.244821e+07
2,2025-01-02,PF,4.961460e+07,162482,305.354411,Thursday,3.457857e+07
3,2025-01-02,PJ,1.067998e+08,1010708,105.668258,Thursday,6.962398e+07
4,2025-01-03,PF,4.921404e+07,191736,256.676046,Friday,3.945706e+07


In [4]:

# =========================
# 3) Salvar kpis_gold.csv
# =========================
OUT_DIR.mkdir(parents=True, exist_ok=True)
daily_kpis.to_csv(OUT_FILE, index=False)
print(f"✅ Arquivo salvo em: {OUT_FILE.resolve()}")


✅ Arquivo salvo em: C:\Users\lucas\OneDrive\Área de Trabalho\Projetos\CloudWalk-Case\streamlit\data\kpis_gold.csv



## 📌 Observações
- O arquivo `kpis_gold.csv` contém: `date`, `entity`, `TPV`, `Transactions`, `Average_Ticket`, `weekday`, `TPV_MA7`.
- O app **Streamlit** deve agora conseguir filtrar **PF / PJ / Ambos** e recalcular os alertas por dia da semana após aplicar os filtros.
- Para adicionar **outras dimensões** (ex.: `product`, `payment_method`, `price_tier`, `installments`), inclua-as na lista `group_dims` e reexecute a célula de agregação.
